In [1]:
# Для нейросети
from tensorflow.keras import utils # Для работы с категориальными данными
from tensorflow.keras.models import Sequential # Полносвязная модель
import matplotlib.pyplot as plt
import os
from tensorflow.keras.layers import Dense, Dropout, SpatialDropout1D, BatchNormalization, Flatten, Activation, Embedding, SimpleRNN, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pandas as pd
import numpy as np
%matplotlib inline

from google.colab import drive
import numpy as np

In [2]:
# Перед пользованием этого ноутбука, вам надо сходить в директорию 
# /Future_Foundation/Up_Great_Сколково/Colabs_to_create_datasets/.
# Берём от туда два файлика с расширением .npy, делаем копию к себе в корневую папку 
# гугл диска, она же "Мой диск", и указываем ниже названия этих файликов!

trainWordIndexesFilename = "Копия trainWordIndexes.npy"
testWordIndexesFilename = "Копия testWordIndexes.npy"

drive.mount('drive')
main_dir = '/content/drive/MyDrive/'

trainWordIndexes = np.load(main_dir + trainWordIndexesFilename, allow_pickle=True)
testWordIndexes = np.load(main_dir + testWordIndexesFilename, allow_pickle=True)

Mounted at drive


FileNotFoundError: ignored

In [ ]:
def getSetFromIndexes(wordIndexes, xLen, step):
  xText = []
  wordsLen = len(wordIndexes) # Считаем количество слов
  index = 0 # Задаем начальный индекс 

  while (index + xLen <= wordsLen): # Идём по всей длине вектора индексов
    xText.append(wordIndexes[index:index+xLen]) # "Откусываем" векторы длины xLen
    index += step # Смещаемся вперёд на step
    
  return xText


# Формирование обучающей и проверочной выборки для каждого класса
# wordIndexes - массив индексов
# xLen - размер окна
# step - шаг окна

def createSetsMultiClasses(wordIndexes, xLen, step): # Функция принимает последовательность индексов, размер окна, шаг окна
  nClasses = len(wordIndexes) # Количество классов
  classesXSamples = []        # Здесь будет список размером "кол-во классов*кол-во окон в тексте*длину окна"
  for wI in wordIndexes:      # Для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step))

  # Формируем один общий xSamples
  xSamples = []
  ySamples = []
  
  for t in range(nClasses):
    xT = classesXSamples[t]
    for i in range(len(xT)): # Перебираем каждое окно определенного класса
      xSamples.append(xT[i]) # Добавляем в общий список выборки
      ySamples.append(utils.to_categorical(t, nClasses)) # Добавляем соответствующий вектор класса

  xSamples = np.array(xSamples)
  ySamples = np.array(ySamples)

  return (xSamples, ySamples)

In [ ]:
xLen = 50 # Размер окна (количество слов в векторе)
step = 6 # Шаг разбиения текста на векторы

In [ ]:
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step)
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, xLen)
xVal, yVal = xTrain[:10000], yTrain[:10000]
xTrain, yTrain = xTrain[10000:], yTrain[10000:]
xBuffer, yBuffer = xTest, yTest
xTest, yTest = xVal, yVal
xVal, yVal = xBuffer, yBuffer
# для каждого "окна" класса мы в соответствие ставим значение true или false => поэтому длины x и y одинаковые
print("Размерности тренировочного набора")
print(xTrain.shape)
print(yTrain.shape)
print()
print("Размерности тестового набора")
print(xTest.shape)
print(yTest.shape)
print()
print("Размерности val набора")
print(xVal.shape)
print(yVal.shape)

In [ ]:
numWords = 20000

In [ ]:
modelLSTM = Sequential()

modelLSTM.add(Embedding(numWords, 100, input_length=xLen))
modelLSTM.add(SpatialDropout1D(0.2))
modelLSTM.add(BatchNormalization())

modelLSTM.add(LSTM(10))
modelLSTM.add(Dense(2, activation='softmax'))

modelLSTM.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

modelLSTM.summary()

In [ ]:
# Обучаем сеть
Batch_Size = 256

historyLSTM = modelLSTM.fit(xTrain, 
                            yTrain, 
                            epochs=15,
                            batch_size=Batch_Size,
                            # validation_data=(xVal, yVal),
                            use_multiprocessing=True)